In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from Functions.visualization_functions import *

In [ ]:
# TODO change the path to the csv files
# Runs to compare
df_ppo = pd.read_csv('ppo.csv')
df_a2c = pd.read_csv('a2c.csv')
df_dqn = pd.read_csv('dqn.csv')
df_heuristic = pd.read_csv('heuristic.csv')
df_greedy = pd.read_csv('greedy.csv')

# Dataframes to compare
df_rl = [df_ppo, df_a2c, df_dqn]
df_all = [df_ppo, df_a2c, df_dqn, df_heuristic, df_greedy]

# Labels for the plots
label_rl = ['PPO', 'A2C', 'DQN']
labels_al = ['PPO', 'A2C', 'DQN', 'Heuristic', 'Greedy']

In [ ]:
benchmark_plot(df_rl, label_rl)

In [ ]:
benchmark_plot(df_all, labels_al)